En este archivo vamos a entrenar una red neuronal para predecir el resultado de los partidos de la Champions bsándonos en datos de los último 7 años.

Para empezar primero cargamos las librerias necesarias.

In [91]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.utils import to_categorical
from keras.layers import Dropout
from keras.optimizers import Adam
from keras import regularizers
import matplotlib.pyplot as plt
import random

Ahora vamos a empezar el entrenamiento para predecir resultados, pero antes de ello vamos a transformar las columnas string de equipos en numéricas para poder evaluarlas y analizarlas durante el procesamiento de la red neuronal.

In [92]:
# Cargar los datos del ranking UEFA
uefa_ranking = pd.read_csv("data/UEFA_Ranking.csv")

#Saca el número de valores unicos de la columna Club de ranking UEFA
uefa_ranking['Club'].nunique()

570

In [93]:
# Crear un diccionario para mapear nombres de equipos a números únicos
team_mapping = {team: idx + 1 for idx, team in enumerate(uefa_ranking['Club'])}

# Cargar los datos de los partidos de la Champions League 2017
champions_data = pd.read_csv("data/champions-league-2017.csv")

# Crear una columna Home Team Name que sea una copia de la columna Home Team
champions_data['Home Team Name'] = champions_data['Home Team']

# Crear una columna Away Team Name que sea una copia de la columna Away Team
champions_data['Away Team Name'] = champions_data['Away Team']

# Reemplazar nombres de equipos por números en las columnas correspondientes
champions_data['Home Team'] = champions_data['Home Team'].map(team_mapping)
champions_data['Away Team'] = champions_data['Away Team'].map(team_mapping)
champions_data['Winner'] = champions_data['Winner'].map(team_mapping)

Vamos a comprobar que todos los datos están bien.

In [94]:
# Ver las primeras filas de los datos
print(champions_data.head())

   Match Number Round Number  Home Team  Away Team Result  Winner  \
0             1            1         20         62  1 - 2      62   
1             2            1         11         26  3 - 0      11   
2            13            1          2         53  3 - 0       2   
3            14            1         50          8  0 - 5       8   
4            25            1          9         66  6 - 0       9   

   Home Team Goals  Away Team Goals  Match Goals     Home Team Name  \
0                1                2            3            Benfica   
1                3                0            3  Manchester United   
2                3                0            3     Bayern München   
3                0                5            5             Celtic   
4                6                0            6            Chelsea   

        Away Team Name  
0          CSKA Moscow  
1             FC Basel  
2           Anderlecht  
3  Paris Saint-Germain  
4           Qarabag FK  


In [95]:
# analizar los datos
print(champions_data.describe())

       Match Number   Home Team   Away Team      Winner  Home Team Goals  \
count    125.000000  125.000000  125.000000  125.000000       125.000000   
mean      63.000000   25.432000   25.464000   17.312000         1.784000   
std       36.228442   28.471076   28.445631   22.202302         1.532361   
min        1.000000    1.000000    1.000000    1.000000         0.000000   
25%       32.000000    6.000000    6.000000    4.000000         1.000000   
50%       63.000000   14.000000   14.000000   10.000000         1.000000   
75%       94.000000   36.000000   36.000000   23.000000         3.000000   
max      125.000000  120.000000  120.000000  120.000000         7.000000   

       Away Team Goals  Match Goals  
count       125.000000   125.000000  
mean          1.424000     3.208000  
std           1.398525     1.622901  
min           0.000000     0.000000  
25%           0.000000     2.000000  
50%           1.000000     3.000000  
75%           2.000000     4.000000  
max        

In [96]:
# ver los datos nulos
print(champions_data.isnull().sum())

Match Number       0
Round Number       0
Home Team          0
Away Team          0
Result             0
Winner             0
Home Team Goals    0
Away Team Goals    0
Match Goals        0
Home Team Name     0
Away Team Name     0
dtype: int64


Como podemos ver está todo bien. Para más información con respecto al CSV que estamos tratando dirigirse a graficas.ipynb

Ahora vamos a comenzar el proceso de entrenamiento de la red neuronal

In [97]:
'''
# Preprocesamiento de datos
X = champions_data[['Home Team', 'Away Team', 'Home Team Goals', 'Away Team Goals']].values
y = champions_data['Winner'].values

# Separar características categóricas y numéricas
categorical_features = ['Home Team', 'Away Team']
numeric_features = ['Home Team Goals', 'Away Team Goals']

X_categorical = champions_data[categorical_features].values
X_numeric = champions_data[numeric_features].values

# Codificar características categóricas
X_categorical_encoded = np.zeros_like(X_categorical)
for i in range(X_categorical.shape[1]):
    label_encoder = LabelEncoder()
    X_categorical_encoded[:, i] = label_encoder.fit_transform(X_categorical[:, i])

# Normalizar características numéricas
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Combinar características categóricas y numéricas
X_combined = np.concatenate((X_categorical_encoded, X_numeric_scaled), axis=1)

# Codificar etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(champions_data['Winner'])
num_classes = len(label_encoder.classes_)

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Normalizar los datos
X_train = X_train / np.max(X_train, axis=0)
X_test = X_test / np.max(X_test, axis=0)

# Convertir etiquetas a one-hot encoding
y_train = to_categorical(y_train, num_classes=num_classes)
y_test = to_categorical(y_test, num_classes=num_classes)

# Definir modelo de red neuronal con más complejidad
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))

# Compilar modelo con hiperparámetros ajustados
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Entrenar modelo
model.fit(X_train, y_train, epochs=100, batch_size=40, validation_split=0.2)

# Evaluar modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
'''

'\n# Preprocesamiento de datos\nX = champions_data[[\'Home Team\', \'Away Team\', \'Home Team Goals\', \'Away Team Goals\']].values\ny = champions_data[\'Winner\'].values\n\n# Separar características categóricas y numéricas\ncategorical_features = [\'Home Team\', \'Away Team\']\nnumeric_features = [\'Home Team Goals\', \'Away Team Goals\']\n\nX_categorical = champions_data[categorical_features].values\nX_numeric = champions_data[numeric_features].values\n\n# Codificar características categóricas\nX_categorical_encoded = np.zeros_like(X_categorical)\nfor i in range(X_categorical.shape[1]):\n    label_encoder = LabelEncoder()\n    X_categorical_encoded[:, i] = label_encoder.fit_transform(X_categorical[:, i])\n\n# Normalizar características numéricas\nscaler = StandardScaler()\nX_numeric_scaled = scaler.fit_transform(X_numeric)\n\n# Combinar características categóricas y numéricas\nX_combined = np.concatenate((X_categorical_encoded, X_numeric_scaled), axis=1)\n\n# Codificar etiquetas\nlab

In [98]:
# Codificar los nombres de los equipos
label_encoder = LabelEncoder()
champions_data['Home Team'] = label_encoder.fit_transform(champions_data['Home Team'])
champions_data['Away Team'] = label_encoder.transform(champions_data['Away Team'])

# Crear una columna para representar el resultado del partido
champions_data['Home is Winner'] = champions_data['Home Team Goals'] - champions_data['Away Team Goals']
champions_data['Home is Winner'] = champions_data['Home is Winner'].apply(lambda x: 1 if x > 0 else 0)  # 1 si el equipo local ganó, 0 si no

# Dividir los datos en características (X) y variable objetivo (y)
X = champions_data[['Home Team', 'Away Team']]
y = champions_data['Home is Winner']
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)
'''
# Preprocesamiento de datos
X = champions_data[['Home Team', 'Away Team']].values
y = champions_data['Winner'].values

# Codificar etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
num_classes = len(label_encoder.classes_)
y = to_categorical(y, num_classes=num_classes)
'''

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir modelo de red neuronal
model = Sequential()
model.add(Dense(128, input_dim=2, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

# Compilar modelo
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.005), metrics=['accuracy'])

# Entrenar modelo
model.fit(X_train, y_train, epochs=90, batch_size=42, validation_data=(X_test, y_test))

# Evaluar modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/90


c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 0.6782 - loss: 0.6544 - val_accuracy: 0.6800 - val_loss: 0.8409
Epoch 2/90
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.6774 - loss: 0.7003 - val_accuracy: 0.6000 - val_loss: 0.7162
Epoch 3/90
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6812 - loss: 0.6063 - val_accuracy: 0.6800 - val_loss: 0.6928
Epoch 4/90
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6535 - loss: 0.5636 - val_accuracy: 0.6400 - val_loss: 0.8137
Epoch 5/90
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.7660 - loss: 0.5202 - val_accuracy: 0.6000 - val_loss: 0.7083
Epoch 6/90
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6176 - loss: 0.6579 - val_accuracy: 0.6400 - val_loss: 0.6304
Epoch 7/90
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7161 - loss: 0.5857 - val_accuracy: 0.6400 - val_loss: 0.6719
Epoch 8/90
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.6943 - loss: 0.6345 - val_accuracy: 0.6000 - val_loss: 0.7329
Epoch 9/90


Vemos que el entrenamiento inicial ha ido sin problemas. Vamos a probar a ver que predicciones realizaría nuestra red neuronal en base a los partidos de la Champions utilizada para obtener una medida de precisión real. Para ello primero vamos a medir la precisión de una predicción en general.

In [99]:
# Hacer predicciones en los datos de prueba
predictions = model.predict(X_test)

# Redondear las predicciones a etiquetas binarias (0 o 1)
predicted_labels = (predictions > 0.5).astype(int)

# Calcular la precisión
accuracy = np.mean(predicted_labels.flatten() == y_test)
print("Accuracy on test data:", accuracy)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Accuracy on test data: 0.64


Como puede verse con este resultado y la medición general anterior, la precisión del modelo es casi inexistente debido al alto error, por lo que vamos a seguir entrenándolo. Primero vamos a aplicar ingeniería de características para darle al modelo más variables sobre las que extraer información. Algunas de estas nos serán útiles a la hora de predecir el resultado de cada partido:

In [100]:
# Calcular la diferencia de goles entre los equipos
champions_data['Goal Difference'] = champions_data['Home Team Goals'] - champions_data['Away Team Goals']

# Calcular la proporción de goles marcados por cada equipo en relación con el total de goles en el partido
champions_data['Home Goals Ratio'] = champions_data['Home Team Goals'] / champions_data['Match Goals']
champions_data['Away Goals Ratio'] = champions_data['Away Team Goals'] / champions_data['Match Goals']

# Fusionar los datos de champions_data con la información de uefa_ranking basado en el nombre del equipo local
champions_data = champions_data.merge(uefa_ranking[['Club', '17/18']], left_on='Home Team Name', right_on='Club', how='left')
champions_data.rename(columns={'17/18': 'Home Team Evaluation'}, inplace=True)
champions_data.drop(columns=['Club'], inplace=True)

# Fusionar los datos de champions_data con la información de uefa_ranking basado en el nombre del equipo visitante
champions_data = champions_data.merge(uefa_ranking[['Club', '17/18']], left_on='Away Team Name', right_on='Club', how='left')
champions_data.rename(columns={'17/18': 'Away Team Evaluation'}, inplace=True)
champions_data.drop(columns=['Club'], inplace=True)

# Para las columnas Result '0 - 0' se añaden 2 columnas: Penalties y Penalties Winner
champions_data['Penalties'] = champions_data['Result'].apply(lambda x: 1 if x == '0 - 0' else 0)
champions_data['Penalties Winner'] = champions_data.apply(lambda row: row['Winner'] if row['Penalties'] == 1 else None, axis=1)

# Ver las primeras filas de los datos
print(champions_data.head())


   Match Number Round Number  Home Team  Away Team Result  Winner  \
0             1            1         15         27  1 - 2      62   
1             2            1         10         19  3 - 0      11   
2            13            1          1         26  3 - 0       2   
3            14            1         25          7  0 - 5       8   
4            25            1          8         28  6 - 0       9   

   Home Team Goals  Away Team Goals  Match Goals     Home Team Name  \
0                1                2            3            Benfica   
1                3                0            3  Manchester United   
2                3                0            3     Bayern München   
3                0                5            5             Celtic   
4                6                0            6            Chelsea   

        Away Team Name  Home is Winner  Goal Difference  Home Goals Ratio  \
0          CSKA Moscow               0               -1          0.333333   
1   

In [101]:
# Ver las estadísticas de los datos
print(champions_data.describe())

       Match Number   Home Team   Away Team      Winner  Home Team Goals  \
count    125.000000  125.000000  125.000000  125.000000       125.000000   
mean      63.000000   13.536000   13.568000   17.312000         1.784000   
std       36.228442    9.400192    9.360466   22.202302         1.532361   
min        1.000000    0.000000    0.000000    1.000000         0.000000   
25%       32.000000    5.000000    5.000000    4.000000         1.000000   
50%       63.000000   13.000000   13.000000   10.000000         1.000000   
75%       94.000000   21.000000   21.000000   23.000000         3.000000   
max      125.000000   31.000000   31.000000  120.000000         7.000000   

       Away Team Goals  Match Goals  Home is Winner  Goal Difference  \
count       125.000000   125.000000       125.00000       125.000000   
mean          1.424000     3.208000         0.44800         0.360000   
std           1.398525     1.622901         0.49929         2.444216   
min           0.000000     

In [102]:
# Ver los datos nulos
print(champions_data.isnull().sum())

Match Number              0
Round Number              0
Home Team                 0
Away Team                 0
Result                    0
Winner                    0
Home Team Goals           0
Away Team Goals           0
Match Goals               0
Home Team Name            0
Away Team Name            0
Home is Winner            0
Goal Difference           0
Home Goals Ratio          7
Away Goals Ratio          7
Home Team Evaluation      0
Away Team Evaluation      0
Penalties                 0
Penalties Winner        118
dtype: int64


In [103]:
# Ver las columnas Home Goals Ratio y Away Goals Ratio donde hay datos nulos
print(champions_data[['Home Goals Ratio', 'Away Goals Ratio']], '\n')
print(champions_data[champions_data['Home Goals Ratio'].isnull()])

     Home Goals Ratio  Away Goals Ratio
0            0.333333          0.666667
1            1.000000          0.000000
2            1.000000          0.000000
3            0.000000          1.000000
4            1.000000          0.000000
..                ...               ...
120          0.714286          0.285714
121          0.333333          0.666667
122          0.500000          0.500000
123          0.666667          0.333333
124          0.750000          0.250000

[125 rows x 2 columns] 

     Match Number          Round Number  Home Team  Away Team Result  Winner  \
5              26                     1         13          6  0 - 0       7   
40             29                     3         28          6  0 - 0       7   
55             44                     4         23          2  0 - 0       3   
78             45                     5          5          2  0 - 0       3   
102           103  Round of 16 - Game 1          9         10  0 - 0      11   
104           

Vemos aquí el problema de partidos donde no hay goles. De momento vamos a dejarlo como está para darle más realismo al entrenamiento de la red neuronal:

In [104]:
# Codificar los nombres de los equipos
label_encoder = LabelEncoder()
champions_data['Home Team'] = label_encoder.fit_transform(champions_data['Home Team'])
champions_data['Away Team'] = label_encoder.transform(champions_data['Away Team'])

# Preprocesamiento de datos con nuevas características
X = champions_data[['Home Team', 'Away Team', 'Home Team Evaluation', 'Away Team Evaluation', 'Home Team Goals', 'Away Team Goals', 'Home Goals Ratio', 'Away Goals Ratio', 'Penalties', 'Penalties Winner']].values
y = champions_data['Home is Winner']
'''
# Separar características categóricas y numéricas
categorical_features = ['Home Team', 'Away Team']
numeric_features = ['Home Team Evaluation', 'Away Team Evaluation', 'Home Team Goals', 'Away Team Goals', 'Home Goals Ratio', 'Away Goals Ratio', 'Penalties', 'Penalties Winner']

X_categorical = champions_data[categorical_features].values
X_numeric = champions_data[numeric_features].values

# Codificar características categóricas
X_categorical_encoded = np.zeros_like(X_categorical)
for i in range(X_categorical.shape[1]):
    label_encoder = LabelEncoder()
    X_categorical_encoded[:, i] = label_encoder.fit_transform(X_categorical[:, i])

# Normalizar características numéricas
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Combinar características categóricas y numéricas
X_combined = np.concatenate((X_categorical_encoded, X_numeric_scaled), axis=1)

# Codificar etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(champions_data['Winner'])
num_classes = len(label_encoder.classes_)
y = to_categorical(y, num_classes=num_classes)'''

print("Número de muestras en X:", X.shape[0])
print("Número de muestras en y:", y.shape[0])

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

# Normalizar los datos
X_train = X_train / np.max(X_train, axis=0)
X_test = X_test / np.max(X_test, axis=0)


Número de muestras en X: 125
Número de muestras en y: 125


Ahora vamos a ampliar el alcance de nuestra red neuronal:

In [105]:
# Definir modelo de red neuronal con más complejidad
model = Sequential()
model.add(Dense(256, input_shape=(X_train.shape[1],), activation='tanh', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

# Compilar modelo con hiperparámetros ajustados
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# Entrenar modelo
model.fit(X_train, y_train, epochs=100, batch_size=40, validation_split=0.2)

# Evaluar modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Epoch 1/100


c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\keras\src\losses\losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(40, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 186ms/step - accuracy: 0.5417 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5583 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 3/100


c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\keras\src\losses\losses.py:22: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5333 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5583 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.5500 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5667 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5667 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.5667 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 9/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.5667 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 10/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.5583 - loss: nan - val_accuracy: 0.6000 - val_loss: nan
Epoch 11/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step 

La precisión ahora está mucho mejor, pero vamos a ver si podemos seguir mejorando. Para ello, vamos a ir probando diferentes parámetros para ajustar la red neuronal:

In [106]:
# Codificar los nombres de los equipos
label_encoder = LabelEncoder()
champions_data['Home Team'] = label_encoder.fit_transform(champions_data['Home Team'])
champions_data['Away Team'] = label_encoder.transform(champions_data['Away Team'])

# Definir características y variable objetivo
X = champions_data[['Home Team', 'Away Team', 'Home Team Evaluation', 'Away Team Evaluation',
          'Home Team Goals', 'Away Team Goals', 'Home Goals Ratio', 'Away Goals Ratio',
          'Penalties', 'Penalties Winner']]
y = champions_data['Home is Winner']

'''

# Preprocesamiento de datos con nuevas características
X = champions_data[['Home Team', 'Away Team', 'Home Team Evaluation', 'Away Team Evaluation', 'Home Team Goals', 'Away Team Goals', 'Home Goals Ratio', 'Away Goals Ratio', 'Penalties', 'Penalties Winner']].values
y = champions_data['Winner'].values

# Separar características categóricas y numéricas
categorical_features = ['Home Team', 'Away Team']
numeric_features = ['Home Team Evaluation', 'Away Team Evaluation', 'Home Team Goals', 'Away Team Goals', 'Home Goals Ratio', 'Away Goals Ratio', 'Penalties', 'Penalties Winner']

X_categorical = champions_data[categorical_features].values
X_numeric = champions_data[numeric_features].values

# Codificar características categóricas
X_categorical_encoded = np.zeros_like(X_categorical)
for i in range(X_categorical.shape[1]):
    label_encoder = LabelEncoder()
    X_categorical_encoded[:, i] = label_encoder.fit_transform(X_categorical[:, i])

# Normalizar características numéricas
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Combinar características categóricas y numéricas
X_combined = np.concatenate((X_categorical_encoded, X_numeric_scaled), axis=1)

# Codificar etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(champions_data['Winner'])
num_classes = len(label_encoder.classes_)
print("Número de clases:", num_classes)
y = to_categorical(y, num_classes=num_classes)

'''

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32)

ajustes = []
capas = ["softmax", "relu", "sigmoid", "tanh", "linear"]
optimizador = ["SGD", "RMSprop", "Adam", "AdamW", "Adadelta",
               "Adagrad", "Adamax", "Adafactor", "Nadam", "Ftrl", "Lion"]

num_capas = [1, 2, 3, 4, 5]
num_neuronas = [8, 16, 32, 64, 128]
learning_rate = [0.1, 0.01, 0.001, 0.0001, 0.00001]

target_accuracy = 0.7
target_loss = 0.3

for _ in range(30):
    # Configuración aleatoria
    configuracion_aleatoria = {
        "capa_activacion": random.choice(capas),
        "optimizador": random.choice(optimizador),
        "num_capas": random.choice(num_capas),
        "num_neuronas": random.choice(num_neuronas),
        "learning_rate": random.choice(learning_rate)
    }

    # Crear el modelo con la configuración aleatoria
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(
        configuracion_aleatoria["num_neuronas"], input_dim=10, activation=configuracion_aleatoria["capa_activacion"]))
    for _ in range(configuracion_aleatoria["num_capas"] - 1):
        model.add(tf.keras.layers.Dense(
            configuracion_aleatoria["num_neuronas"], activation=configuracion_aleatoria["capa_activacion"]))

    # Última capa ajustada para el problema de clasificación multiclase
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    # Compilar el modelo con el optimizador y learning rate aleatorio
    optimizer = getattr(tf.keras.optimizers, configuracion_aleatoria["optimizador"])(
        learning_rate=configuracion_aleatoria["learning_rate"])
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo
    history = model.fit(X_train, y_train, epochs=70, batch_size=64)

    # Evaluar modelo
    loss, accuracy = model.evaluate(X_test, y_test)
    print("Test Loss:", loss)
    print("Test Accuracy:", accuracy)


    # Verificar si se alcanzaron las condiciones de término
    if accuracy >= target_accuracy and 0 <= loss <= target_loss:
        break

    # Si no se alcanzaron las condiciones de término, se guarda la configuración y se ordena por precisión en orden ascendente y evitando duplicados
    ajustes.append((configuracion_aleatoria, accuracy, loss))
    print(ajustes)

    # Guardar en un archivo txt
    with open('ajustes.txt', 'w') as f:
        for item in ajustes:
            f.write("%s\n" % str(item))

Epoch 1/70


c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5608 - loss: 0.6931  
Epoch 2/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5452 - loss: 0.6931 
Epoch 3/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5608 - loss: 0.6931 
Epoch 4/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5504 - loss: 0.6931 
Epoch 5/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5504 - loss: 0.6931 
Epoch 6/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5713 - loss: 0.6931 
Epoch 7/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5556 - loss: 0.6931 
Epoch 8/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5660 - loss: 0.6931 
Epoch 9/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5713 - loss: 0.6931 
Epoch 10/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5452 - loss: 0.6931 
Epoch 11/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5608 - loss: 0.6931 
Epoch 12/70
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5504 - loss: 0.6931 
Epoch 13/70

Vamos a continuar añadiéndo datos de entrenamiento. Para ello vamos a prepararlos primero:

In [16]:
champions_data2 = pd.read_csv("data/champions-league-2018.csv")
champions_data3 = pd.read_csv("data/champions-league-2019.csv")
champions_data4 = pd.read_csv("data/champions-league-2020.csv")
champions_data5 = pd.read_csv("data/champions-league-2021.csv")
champions_data6 = pd.read_csv("data/champions-league-2022.csv")

# Reemplazar nombres de equipos por números en las columnas correspondientes
champions = [champions_data2, champions_data3, champions_data4, champions_data5, champions_data6]
for data in champions:
    data['Home Team'] = data['Home Team'].map(team_mapping)
    data['Away Team'] = data['Away Team'].map(team_mapping)
    data['Winner'] = data['Winner'].map(team_mapping)
    # Calcular la diferencia de goles entre los equipos
    data['Goal Difference'] = data['Home Team Goals'] - data['Away Team Goals']

    # Calcular la proporción de goles marcados por cada equipo en relación con el total de goles en el partido
    data['Home Goals Ratio'] = data['Home Team Goals'] / data['Match Goals']
    data['Away Goals Ratio'] = data['Away Team Goals'] / data['Match Goals']

In [17]:
# Ver los datos nulos de cada conjunto de datos
for i, data in enumerate(champions):
    print(f"Data {i + 2} null values:\n{data.isnull().sum()}\n")

Data 2 null values:
Match Number        0
Round Number        0
Home Team           0
Away Team           0
Result              0
Winner              0
Home Team Goals     0
Away Team Goals     0
Match Goals         0
Goal Difference     0
Home Goals Ratio    7
Away Goals Ratio    7
dtype: int64

Data 3 null values:
Match Number        0
Round Number        0
Home Team           0
Away Team           0
Result              0
Winner              0
Home Team Goals     0
Away Team Goals     0
Match Goals         0
Goal Difference     0
Home Goals Ratio    4
Away Goals Ratio    4
dtype: int64

Data 4 null values:
Match Number        0
Round Number        0
Home Team           0
Away Team           0
Result              0
Winner              0
Home Team Goals     0
Away Team Goals     0
Match Goals         0
Goal Difference     0
Home Goals Ratio    6
Away Goals Ratio    6
dtype: int64

Data 5 null values:
Match Number        0
Round Number        0
Home Team           0
Away Team           

Vemos que el csv 2 y 3 tienen datos nulos en partidos. Vamos a visualizarlos:

In [18]:
#Ver las filas Home Team, Away Team y Winner donde hay datos nulos en champions_data2
# Seleccionar las filas donde hay valores nulos en las columnas específicas
null_data = champions_data2[champions_data2[['Home Team', 'Away Team', 'Winner']].isnull().any(axis=1)]

# Mostrar las filas seleccionadas
print(null_data[['Home Team', 'Away Team', 'Winner']])

Empty DataFrame
Columns: [Home Team, Away Team, Winner]
Index: []


Las filas de Home Team donde no se ha podido dar un número al equipo de champions_data2 son la 4, 49 y 86. De Away Team la 24, 44 y 77 y de Winner la 49 que como ya ha salido se descarta. Para solucionar esto vamos a introducir en UEFA_Ranking.csv esos equipos al final y actualizar el mapeo. Para ello buscamos en data/champions-league-2018.csv esas filas, extraemos el nombre del equipo de la columna correspondiente y lo añadimos a UEFA_Ranking con todos los campos 0 excepto el de Position que tendrá el mismo número + 1 que el de la fila anterior última. Después volvemos a mapear:

In [19]:
#Ver la fila 4, 24, 44, 49, 77, 86 de champions-league-2018.csv
league_2018 = pd.read_csv("data/champions-league-2018.csv")
print(league_2018.iloc[[4, 24, 44, 49, 77, 86]])

    Match Number Round Number            Home Team            Away Team  \
4              5            1        Crvena zvezda               Napoli   
24            25            2  Paris Saint-Germain        Crvena zvezda   
44            45            3            Liverpool        Crvena zvezda   
49            50            4        Crvena zvezda            Liverpool   
77            78            5               Napoli        Crvena zvezda   
86            87            6        Crvena zvezda  Paris Saint-Germain   

   Result               Winner  Home Team Goals  Away Team Goals  Match Goals  
4   0 - 0               Napoli                0                0            0  
24  6 - 1  Paris Saint-Germain                6                1            7  
44  4 - 0            Liverpool                4                0            4  
49  2 - 0        Crvena zvezda                2                0            2  
77  3 - 1               Napoli                3                1          

El problema es el equipo Crvena zvezda. Vamos a añadirlo al mapeo una única vez:

In [20]:
# Agregar una fila para el equipo "Crvena zvezda" al archivo UEFA_Ranking.csv
new_row = {'Club': 'Crvena zvezda'}

if 'Crvena zvezda' not in uefa_ranking['Club'].values:
    uefa_ranking = pd.concat([uefa_ranking, pd.DataFrame([new_row])], ignore_index=True)

    # Guardar el DataFrame actualizado en un nuevo archivo CSV
    uefa_ranking.to_csv("data/UEFA_Ranking.csv", index=False)

In [21]:
# Actualizar el mapeo de nombres de equipos a números únicos
team_mapping = {team: idx + 1 for idx, team in enumerate(uefa_ranking['Club'])}

champions_data2 = pd.read_csv("data/champions-league-2018.csv")
champions_data3 = pd.read_csv("data/champions-league-2019.csv")
champions_data4 = pd.read_csv("data/champions-league-2020.csv")
champions_data5 = pd.read_csv("data/champions-league-2021.csv")
champions_data6 = pd.read_csv("data/champions-league-2022.csv")

# Reemplazar nombres de equipos por números en las columnas correspondientes
champions = [champions_data2, champions_data3, champions_data4, champions_data5, champions_data6]
for data in champions:
    data['Home Team'] = data['Home Team'].map(team_mapping)
    data['Away Team'] = data['Away Team'].map(team_mapping)
    data['Winner'] = data['Winner'].map(team_mapping)
    # Calcular la diferencia de goles entre los equipos
    data['Goal Difference'] = data['Home Team Goals'] - data['Away Team Goals']

    # Calcular la proporción de goles marcados por cada equipo en relación con el total de goles en el partido
    data['Home Goals Ratio'] = data['Home Team Goals'] / data['Match Goals']
    data['Away Goals Ratio'] = data['Away Team Goals'] / data['Match Goals']

In [22]:
# Ver los datos nulos de cada conjunto de datos
for i, data in enumerate(champions):
    print(f"Data {i + 2} null values:\n{data.isnull().sum()}\n")

Data 2 null values:
Match Number        0
Round Number        0
Home Team           0
Away Team           0
Result              0
Winner              0
Home Team Goals     0
Away Team Goals     0
Match Goals         0
Goal Difference     0
Home Goals Ratio    7
Away Goals Ratio    7
dtype: int64

Data 3 null values:
Match Number        0
Round Number        0
Home Team           0
Away Team           0
Result              0
Winner              0
Home Team Goals     0
Away Team Goals     0
Match Goals         0
Goal Difference     0
Home Goals Ratio    4
Away Goals Ratio    4
dtype: int64

Data 4 null values:
Match Number        0
Round Number        0
Home Team           0
Away Team           0
Result              0
Winner              0
Home Team Goals     0
Away Team Goals     0
Match Goals         0
Goal Difference     0
Home Goals Ratio    6
Away Goals Ratio    6
dtype: int64

Data 5 null values:
Match Number        0
Round Number        0
Home Team           0
Away Team           

Vemos que ya no hay valores nulos donde no debe haberlos, así que proseguimos con la unión de los datos:

In [23]:
# Concatenar todos los datos verticalmente
combined_data = pd.concat([champions_data, champions_data2, champions_data3, champions_data4, champions_data5, champions_data6], axis=0, ignore_index=True)

Vamos a analizar el resultado de la unión:

In [24]:
# Ver las filas union de los datos
print(combined_data.head())

   Match Number Round Number  Home Team  Away Team Result  Winner  \
0             1            1         20         62  1 - 2      62   
1             2            1         11         26  3 - 0      11   
2            13            1          2         53  3 - 0       2   
3            14            1         50          8  0 - 5       8   
4            25            1          9         66  6 - 0       9   

   Home Team Goals  Away Team Goals  Match Goals     Home Team Name  \
0                1                2            3            Benfica   
1                3                0            3  Manchester United   
2                3                0            3     Bayern München   
3                0                5            5             Celtic   
4                6                0            6            Chelsea   

        Away Team Name  Goal Difference  Home Goals Ratio  Away Goals Ratio  \
0          CSKA Moscow               -1          0.333333          0.666667   


In [25]:
# Ver la forma de los datos combinados
print(combined_data.shape)

(744, 16)


In [26]:
# Ver los datos
print(combined_data.describe())

       Match Number   Home Team   Away Team      Winner  Home Team Goals  \
count    744.000000  744.000000  744.000000  744.000000       744.000000   
mean      62.538978   36.950269   36.228495   25.051075         1.649194   
std       35.883295   81.404405   79.599597   65.773277         1.433797   
min        1.000000    1.000000    1.000000    1.000000         0.000000   
25%       31.750000    7.000000    7.000000    5.000000         1.000000   
50%       62.500000   17.000000   17.000000   11.000000         1.000000   
75%       93.250000   33.000000   33.000000   23.000000         2.000000   
max      125.000000  570.000000  570.000000  570.000000         7.000000   

       Away Team Goals  Match Goals  Goal Difference  Home Goals Ratio  \
count       744.000000   744.000000       744.000000        704.000000   
mean          1.403226     3.052419         0.245968          0.547807   
std           1.385870     1.723465         2.232149          0.361085   
min           0.000

In [27]:
# Ver los datos nulos
print(combined_data.isnull().sum())

Match Number              0
Round Number              0
Home Team                 0
Away Team                 0
Result                    0
Winner                    0
Home Team Goals           0
Away Team Goals           0
Match Goals               0
Home Team Name          619
Away Team Name          619
Goal Difference           0
Home Goals Ratio         40
Away Goals Ratio         40
Home Team Evaluation    619
Away Team Evaluation    619
dtype: int64


In [28]:
#mostrar las filas con datos nulos
print(combined_data[combined_data.isnull().any(axis=1)])

     Match Number          Round Number  Home Team  Away Team Result  Winner  \
5              26                     1         14          7  0 - 0       7   
40             29                     3         66          7  0 - 0       7   
55             44                     4         41          3  0 - 0       3   
78             45                     5          6          3  0 - 0       3   
102           103  Round of 16 - Game 1         10         11  0 - 0      11   
..            ...                   ...        ...        ...    ...     ...   
739           121             SF Game 1          1          4  1 - 1       4   
740           122             SF Game 1         25         18  0 - 2      18   
741           123             SF Game 2         18         25  1 - 0      18   
742           124             SF Game 2          4          1  4 - 0       4   
743           125                 Final          4         18  1 - 0       4   

     Home Team Goals  Away Team Goals  

In [29]:
# Sacar el numero de datos totales de las columnas ['Home Team', 'Away Team', 'Home Team Goals', 'Away Team Goals', 'Goal Difference', 'Home Goals Ratio', 'Away Goals Ratio', 'Winner']
print(combined_data[['Home Team', 'Away Team', 'Home Team Goals', 'Away Team Goals', 'Goal Difference', 'Home Goals Ratio', 'Away Goals Ratio', 'Winner']].count())

Home Team           744
Away Team           744
Home Team Goals     744
Away Team Goals     744
Goal Difference     744
Home Goals Ratio    704
Away Goals Ratio    704
Winner              744
dtype: int64


In [30]:
# Sacar el numero de datos únicos de las columnas ['Home Team', 'Away Team', 'Home Team Goals', 'Away Team Goals', 'Goal Difference', 'Home Goals Ratio', 'Away Goals Ratio', 'Winner']
print(combined_data[['Home Team', 'Away Team', 'Home Team Goals', 'Away Team Goals', 'Goal Difference', 'Home Goals Ratio', 'Away Goals Ratio', 'Winner']].nunique())

Home Team           72
Away Team           72
Home Team Goals      8
Away Team Goals      9
Goal Difference     15
Home Goals Ratio    23
Away Goals Ratio    23
Winner              65
dtype: int64


In [31]:
# Preprocesamiento de datos
X = combined_data[['Home Team', 'Away Team', 'Goal Difference', 'Home Goals Ratio', 'Away Goals Ratio']].values
y = combined_data['Winner'].values

# Separar características categóricas y numéricas
categorical_features = ['Home Team', 'Away Team']
numeric_features = ['Home Team Goals', 'Away Team Goals', 'Goal Difference', 'Home Goals Ratio', 'Away Goals Ratio']

X_categorical = combined_data[categorical_features].values
X_numeric = combined_data[numeric_features].values

# Codificar características categóricas
X_categorical_encoded = np.zeros_like(X_categorical)
for i in range(X_categorical.shape[1]):
    label_encoder = LabelEncoder()
    X_categorical_encoded[:, i] = label_encoder.fit_transform(X_categorical[:, i])

# Normalizar características numéricas
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Combinar características categóricas y numéricas
X_combined = np.concatenate((X_categorical_encoded, X_numeric_scaled), axis=1)

# Codificar etiquetas
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(combined_data['Winner'])
num_classes = len(label_encoder.classes_)
y = to_categorical(y, num_classes=num_classes)

print("Número de muestras en X:", X.shape[0])
print("Número de muestras en y:", y.shape[0])

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Normalizar los datos
X_train = X_train / np.max(X_train, axis=0)
X_test = X_test / np.max(X_test, axis=0)

# Definir modelo de red neuronal con arquitectura modificada
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Cambiar a 'softmax' y ajustar neuronas

# Compilar y entrenar el modelo...
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=40, validation_split=0.2)

# Evaluar modelo
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

Número de muestras en X: 744
Número de muestras en y: 744


c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100


12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 46ms/step - accuracy: 0.0416 - loss: 4.1726 - val_accuracy: 0.0840 - val_loss: 4.1671
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0558 - loss: 4.1649 - val_accuracy: 0.0840 - val_loss: 4.1599
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0621 - loss: 4.1579 - val_accuracy: 0.0840 - val_loss: 4.1527
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0537 - loss: 4.1504 - val_accuracy: 0.0840 - val_loss: 4.1457
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.0513 - loss: 4.1442 - val_accuracy: 0.0840 - val_loss: 4.1388
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0710 - loss: 4.1374 - val_accuracy: 0.0840 - val_loss: 4.1318
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0613 - loss: 4.1286 - val_accuracy: 0.0840 - val_loss: 4.1249
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0599 - loss: 4.1209 - val_accuracy: 0.0840 - val_loss: 

PRUEBA

In [32]:
# Preprocesamiento de datos
X = combined_data[['Home Team', 'Away Team', 'Goal Difference', 'Home Goals Ratio', 'Away Goals Ratio']].values
y = combined_data[['Winner', 'Home Team Goals', 'Away Team Goals']].values

# Separar características categóricas y numéricas
categorical_features = ['Home Team', 'Away Team']
numeric_features = ['Goal Difference', 'Home Goals Ratio', 'Away Goals Ratio']

X_categorical = combined_data[categorical_features].values
X_numeric = combined_data[numeric_features].values

# Codificar características categóricas
X_categorical_encoded = np.zeros_like(X_categorical)
for i in range(X_categorical.shape[1]):
    label_encoder = LabelEncoder()
    X_categorical_encoded[:, i] = label_encoder.fit_transform(X_categorical[:, i])

# Normalizar características numéricas
scaler = StandardScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

# Combinar características categóricas y numéricas
X_combined = np.concatenate((X_categorical_encoded, X_numeric_scaled), axis=1)

# Codificar etiquetas
label_encoder = LabelEncoder()
y_winner = label_encoder.fit_transform(y[:, 0])
y_home_goals = y[:, 1]
y_away_goals = y[:, 2]

num_winner_classes = len(label_encoder.classes_)

# Convertir etiquetas a formato categórico solo para el resultado del ganador
y_winner = to_categorical(y_winner, num_classes=num_winner_classes)

print("Número de muestras en X:", X.shape[0])
print("Número de muestras en y_winner:", y_winner.shape[0])

# Dividir datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_winner_train, y_winner_test, y_home_goals_train, y_home_goals_test, y_away_goals_train, y_away_goals_test = train_test_split(X_combined, y_winner, y_home_goals, y_away_goals, test_size=0.2, random_state=42)

# Normalizar los datos
X_train = X_train / np.max(X_train, axis=0)
X_test = X_test / np.max(X_test, axis=0)

# Definir modelo de red neuronal con arquitectura modificada
model = Sequential()
model.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))

# Modelo de salida para la predicción del ganador
model.add(Dense(num_winner_classes, activation='softmax', name='winner_output'))

# Compilar y entrenar el modelo...
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_winner_train, epochs=100, batch_size=40, validation_split=0.2)

# Evaluar modelo para predicción del ganador
loss, accuracy = model.evaluate(X_test, y_winner_test)
print("Test Loss (Winner):", loss)
print("Test Accuracy (Winner):", accuracy)

# Preparar el modelo para la predicción de los goles de los equipos
model_goals = Sequential()
model_goals.add(Dense(128, input_shape=(X_train.shape[1],), activation='relu'))
model_goals.add(Dropout(0.5))
model_goals.add(Dense(64, activation='relu'))
model_goals.add(Dropout(0.5))
model_goals.add(Dense(32, activation='relu'))

# Modelo de salida para la predicción de los goles del equipo local
model_goals.add(Dense(1, name='home_goals_output'))

# Compilar y entrenar el modelo...
model_goals.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
model_goals.fit(X_train, y_home_goals_train, epochs=100, batch_size=40, validation_split=0.2)

# Evaluar modelo para predicción de los goles del equipo local
loss, mae = model_goals.evaluate(X_test, y_home_goals_test)
print("Test Loss (Home Goals):", loss)
print("Test MAE (Home Goals):", mae)

# Modelo de salida para la predicción de los goles del equipo visitante
model_goals.add(Dense(1, name='away_goals_output'))

# Compilar y entrenar el modelo...
model_goals.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
model_goals.fit(X_train, y_away_goals_train, epochs=100, batch_size=40, validation_split=0.2)

# Evaluar modelo para predicción de los goles del equipo visitante
loss, mae = model_goals.evaluate(X_test, y_away_goals_test)
print("Test Loss (Away Goals):", loss)
print("Test MAE (Away Goals):", mae)


Número de muestras en X: 744
Número de muestras en y_winner: 744
Epoch 1/100


c:\Users\javie\OneDrive\Documents\GitHub\Champions\venv\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


12/12 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.0690 - loss: 4.1727 - val_accuracy: 0.0672 - val_loss: 4.1670
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0629 - loss: 4.1654 - val_accuracy: 0.0672 - val_loss: 4.1596
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0619 - loss: 4.1576 - val_accuracy: 0.0336 - val_loss: 4.1525
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0389 - loss: 4.1506 - val_accuracy: 0.0672 - val_loss: 4.1454
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0545 - loss: 4.1429 - val_accuracy: 0.0672 - val_loss: 4.1384
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.0613 - loss: 4.1350 - val_accuracy: 0.0672 - val_loss: 4.1313
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.0364 - loss: 4.1272 - val_accuracy: 0.0672 - val_loss: 4.1244
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.0427 - loss: 4.1219 - val_accuracy: 0.0336 - val_loss: 4

Hemos rellenado la red neuronal con mucha más información. Ahora vamos a ver que tal predice el resultado de los partidos basándonos en la información ya existente de los csv: